In [1]:
%matplotlib inline
from sklearn.datasets import load_iris
from sklearn import tree
from sklearn.cross_validation import cross_val_score, train_test_split
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from sklearn.grid_search import RandomizedSearchCV
from sklearn.cross_validation import cross_val_score
from time import time
from operator import itemgetter
from scipy.stats import randint
from sklearn import svm, datasets
from sklearn.preprocessing import StandardScaler
from __future__ import print_function
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
import xlrd

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


# Data Pre-processing

In [2]:
df = pd.read_excel('HW3.xlsx')

In [3]:
X = df.iloc[:,1:len(df.columns)-2]
y = df[["Spending"]]

# purchase = 1
X.pur = X.loc[df['Purchase'] == 1]
y.pur = y.loc[df['Purchase'] == 1]

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


# Spliting dataset into training/testing & Normalization

In [32]:
# separate by training / testing/ validation data (70/20/10)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
x_train2, x_test2, y_train2, y_test2 = train_test_split(X.pur, y.pur, test_size=0.4, random_state=42)

# data normalization
scaler = StandardScaler() 
scaler.fit(X)
scaler.fit(X.pur)

scaler.fit(X[["last_update_days_ago","1st_update_days_ago"]])
scaler.fit(X.pur[["last_update_days_ago","1st_update_days_ago"]])
X[["last_update_days_ago","1st_update_days_ago"]] = scaler.transform(X[["last_update_days_ago","1st_update_days_ago"]])
X.pur[["last_update_days_ago","1st_update_days_ago"]] = scaler.transform(X.pur[["last_update_days_ago","1st_update_days_ago"]])

scaler.fit(x_train[["last_update_days_ago","1st_update_days_ago"]])
scaler.fit(x_train2[["last_update_days_ago","1st_update_days_ago"]])
x_train[["last_update_days_ago","1st_update_days_ago"]] = scaler.transform(x_train[["last_update_days_ago","1st_update_days_ago"]]) 
x_train2[["last_update_days_ago","1st_update_days_ago"]] = scaler.transform(x_train2[["last_update_days_ago","1st_update_days_ago"]]) 

scaler.fit(x_test[["last_update_days_ago","1st_update_days_ago"]])
scaler.fit(x_test2[["last_update_days_ago","1st_update_days_ago"]])
x_test[["last_update_days_ago","1st_update_days_ago"]] = scaler.transform(x_test[["last_update_days_ago","1st_update_days_ago"]])
x_test2[["last_update_days_ago","1st_update_days_ago"]] = scaler.transform(x_test2[["last_update_days_ago","1st_update_days_ago"]])


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-do

# Feature Selection

In [42]:
from sklearn.feature_selection import RFE
from mlxtend.feature_selection import SequentialFeatureSelector as sfs

y_train_val_lst = map(lambda x: x[0], y_train.values)
y_train_val_ser = pd.Series(y_train_val_lst)

# recursive feature elimination (RFE)
estimator = SVR(kernel="linear")
selector = RFE(estimator, 5, step=1)
selector = selector.fit(x_train, y_train_val_ser)
selector.support_

array([False, False, False, False, False, False, False, False,  True,
       False, False, False, False, False, False, False,  True,  True,
        True, False, False,  True])

In [43]:
# selected columns
x_train_selected = x_train[x_train.columns[selector.support_ == True]]
X_selected = X[X.columns[selector.support_ == True]]

# Feature Selection : purchase = 1

In [45]:
#### purchaser = 1
y_train_val_lst = map(lambda x: x[0], y_train2.values)
y_train_val_ser = pd.Series(y_train_val_lst)

estimator = SVR(kernel="linear")
selector = RFE(estimator, 5, step=1)
selector2 = selector.fit(x_train2, y_train_val_ser)
selector.support_

array([ True,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False,  True,  True,
       False, False, False,  True])

In [66]:
# selected columns
x_train2_selected = x_train2[x_train2.columns[selector.support_ == True]]
X2_selected = X[X.columns[selector2.support_ == True]]

# Cross Validation and parameter selection

In [53]:
## Cross Validation Data 

# Set the parameters 
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]


svr = svm.SVR()
clf = GridSearchCV(svr, tuned_parameters, cv=5, verbose=1, scoring='neg_mean_squared_error')
clf.fit(x_train_selected, y_train)

print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print()
print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']

for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
print()

Fitting 5 folds for each of 12 candidates, totalling 60 fits


/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

Best parameters set found on development set:

{'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}

Grid scores on development set:

-0.243 (+/-0.143) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
-0.261 (+/-0.155) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
-0.074 (+/-0.066) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
-0.243 (+/-0.143) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.336 (+/-0.054) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
-0.067 (+/-0.063) for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
0.472 (+/-0.045) for {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}
0.343 (+/-0.051) for {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}
0.441 (+/-0.049) for {'C': 1, 'kernel': 'linear'}
0.466 (+/-0.048) for {'C': 10, 'kernel': 'linear'}
0.467 (+/-0.051) for {'C': 100, 'kernel': 'linear'}
0.468 (+/-0.050) for {'C': 1000, 'kernel': 'linear'}



[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:   12.0s finished
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# Cross Validation and parameter selection: purchase = 1

In [72]:
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]


svr = svm.SVR()
clf = GridSearchCV(svr, tuned_parameters, cv=5, verbose=1)
clf.fit(x_train2_selected, y_train2)

print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print()
print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']

for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
print()

Fitting 5 folds for each of 12 candidates, totalling 60 fits


/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

Best parameters set found on development set:

{'C': 1000, 'kernel': 'linear'}

Grid scores on development set:

-0.048 (+/-0.069) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
-0.052 (+/-0.068) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
-0.010 (+/-0.077) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
-0.048 (+/-0.069) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.186 (+/-0.193) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
-0.009 (+/-0.079) for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
0.318 (+/-0.303) for {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}
0.190 (+/-0.203) for {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}
0.291 (+/-0.266) for {'C': 1, 'kernel': 'linear'}
0.327 (+/-0.317) for {'C': 10, 'kernel': 'linear'}
0.331 (+/-0.321) for {'C': 100, 'kernel': 'linear'}
0.332 (+/-0.324) for {'C': 1000, 'kernel': 'linear'}



/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:    1.2s finished
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# Nested Cross Validation

In [57]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
import numpy as np
import math

print(__doc__)

# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

NUM_TRIALS = 2;
nested_scores= np.zeros(NUM_TRIALS)

for i in range(NUM_TRIALS):
    inner_cv = KFold(n_splits=2, shuffle=True, random_state=i)
    outer_cv = KFold(n_splits=2, shuffle=True, random_state=i)

    # Non_nested parameter search and scoring
    svr = svm.SVR()
    clf = GridSearchCV(estimator=svr, param_grid=tuned_parameters, cv=inner_cv,scoring='neg_mean_squared_error')
    clf.fit(X,y)


# Nested CV with parameter optimization
    nested_score = cross_val_score(clf, x_train_selected, y_train, cv=outer_cv)
    nested_scores[i] = nested_score.mean()
    

print(clf.estimator)
print("Average scores of {0:6f} with std. dev. of {1:6f}.".format(math.sqrt(-nested_scores.mean()), nested_scores.std()))

Automatically created module for IPython interactive environment


/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
Average scores of 141.384582 with std. dev. of 8.601097.


/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Final tuned hyperparameter: {'C': 1, 'kernel': 'rbf'}

# Nested Cross Validation: purchase = 1

In [70]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
import numpy as np
import math

print(__doc__)

# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

NUM_TRIALS = 2;
nested_scores= np.zeros(NUM_TRIALS)

for i in range(NUM_TRIALS):
    inner_cv = KFold(n_splits=2, shuffle=True, random_state=i)
    outer_cv = KFold(n_splits=2, shuffle=True, random_state=i)

    # Non_nested parameter search and scoring
    svr = svm.SVR()
    clf = GridSearchCV(estimator=svr, param_grid=tuned_parameters, cv=inner_cv,scoring='neg_mean_squared_error')
    clf.fit(X,y)


# Nested CV with parameter optimization
    nested_score = cross_val_score(clf, x_train2_selected, y_train2, cv=outer_cv)
    nested_scores[i] = nested_score.mean()
    

print(clf.estimator)
print("Average scores of {0:6f} with std. dev. of {1:6f}.".format(math.sqrt(-nested_scores.mean()), nested_scores.std()))

Automatically created module for IPython interactive environment


/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
Average scores of 167.860354 with std. dev. of 294.461033.


/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

Final tuned hyperparameter: {'C': 1, 'kernel': 'rbf'}

# Prediction ＆ Validation

In [71]:
svc = svm.SVR(kernel='rbf', C=1).fit(x_train, y_train)
scores = cross_val_score(svc, x_train_selected , y_train, cv=5, scoring='neg_mean_squared_error')
print(math.sqrt(-scores.mean()))

svc = svm.SVR(kernel='rbf', C=1).fit(x_train2, y_train2)
scores = cross_val_score(svc, x_train2_selected , y_train2, cv=5, scoring='neg_mean_squared_error')
print(math.sqrt(-scores.mean()))

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

193.43565928223035
205.9057992898983


/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin